In [9]:
import collections

import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
import tensorflow_federated as tff
import pandas as pd
from sklearn.model_selection import train_test_split

import sys 
import matplotlib 
import scipy as sp 
import IPython
from IPython import display 
import sklearn 
import random
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix
from scipy.stats import chi2_contingency

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

TEST_SIZE = 0.2
NUM_CLIENTS = 5
ACTIVE_CLIENTS = 5
BATCH_SIZE = 5
path = os.path.dirname(tff.__file__)
print(path)

-------------------------
/home/ella/Desktop/Progetto/venv-federated/lib/python3.9/site-packages/tensorflow_federated


In [10]:
# Lista delle GPU disponibili
gpus = tf.config.list_physical_devices('GPU')
print("GPUs disponibili: ", gpus)

# Verifica se TensorFlow utilizza la GPU
if gpus:
    print("TensorFlow sta usando la GPU")
else:
    print("TensorFlow non sta usando la GPU")

GPUs disponibili:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow sta usando la GPU


In [11]:
# Import del dataset e divisione in train e test
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Viene diviso il train set in train e validation set
train_df, val_df = train_test_split(train_df, test_size = TEST_SIZE, random_state = 42)

train_x = train_df.iloc[:,1:].to_numpy()
train_y = train_df.iloc[:,0].to_numpy()

val_x = val_df.iloc[:,1:].to_numpy()
val_y = val_df.iloc[:,0].to_numpy()

test_x = test_df.iloc[:,1:].to_numpy()
test_y = test_df.iloc[:,0].to_numpy()

In [12]:
# Overview dei dataset
print(train_df.dtypes)
print(train_df.describe(include = 'all'))

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object
       PassengerId HomePlanet CryoSleep    Cabin  Destination          Age  \
count         6954       6786      6777     6796         6815  6806.000000   
unique        6954          3         2     5441            3          NaN   
top        2513_01      Earth     False  D/176/S  TRAPPIST-1e          NaN   
freq             1       3691      4377        7         4733          NaN   
mean           NaN        NaN       NaN      NaN          NaN    28.828093   
std            NaN        NaN       NaN      NaN          NaN    14.446399   
min            NaN        NaN       NaN      NaN          NaN     0.000000   
25%            NaN        

In [13]:
# Funzione per calcolare la dipendenza tra feature categoriche con il chi-square test
def chi_square(f1, f2):
    # Cross tabulation between GENDER and APPROVE_LOAN
    crosstab_result=pd.crosstab(index=f1,columns=f2)
    print(crosstab_result)

    # Performing Chi-sq test
    test = chi2_contingency(crosstab_result)
    
    # P-Value is the Probability of H0 being True
    # If P-Value&gt;0.05 then only we Accept the assumption(H0)
    
    print('The P-Value of the ChiSq Test is:', test[1])

In [14]:
# Strategia: Home_planet
# Si nota che tutti gli appartenenti allo stesso gruppo hanno lo stesso home planet e quindi per i nan di home planet si può
# assegnare il valore dell'home planet del gruppo (che sarà uguale alla moda). Restano problematici i gruppi composti da una
# sola persona, si introduce quindi una nuova categoria di home planet per persona con origine sconoosciuta.

def mv_hp_d(dataset):
    solo_hp_nan = train_df[(train_df['HomePlanet'].isna()) & (train_df['Solo']==1)].shape[0]
    tot_hp_nan = train_df['HomePlanet'].isna().sum() 
    diff = tot_hp_nan - solo_hp_nan
    print(f"Numero di persone con HomePlanet nullo: {str(tot_hp_nan)}")
    print(f"Numero di gruppi con HomePlanet nullo: {str(diff)}")
    print(f"Numero di persone da sole con HomePlanet nullo: {str(solo_hp_nan)}")

    group_hp = train_df[(train_df['Solo']==0)].groupby('Group')['HomePlanet'].nunique().value_counts()
    print(f"Tutti gli appartenenti allo stesso gruppo hanno stesso home planet? {group_hp.shape[0]==1}")
    chi_square(dataset['Destination'], dataset['Group'])
    
    # HomePlanet
    dataset['HomePlanet'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('Group')['HomePlanet'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['HomePlanet'][(dataset['Solo']==1)] = dataset[(dataset['Solo']==1)].groupby('Group')['HomePlanet'].transform(lambda x: x.fillna("unknown"))
    print(f"Numero di persone con HomePlanet nullo: {str(dataset['HomePlanet'].isna().sum())}")

    # Destination
    dataset['Destination'] = dataset['Destination'].transform(lambda x: x.fillna(x.mode()[0]))
    print(f"Numero di persone con Destination nullo: {str(dataset['Destination'].isna().sum())}")

# Strategia: Age
# Si distinguono 3 casi di persone con Age nullo: persone sole, persone con famiglia che non spendono e persone con famiglia che spendono.
# Per assegnare l'età si usa la mediana del caso di appartenenza, immaginando che è meno probabile che una persona sola sia un bambino
# e che al contrario è più probabile che una persona che non spende, ma è in un gruppo lo sia.
def mv_age(dataset):
    solo = dataset['Solo']==1
    family_no_spending = (dataset['Family_size']>1) & (dataset['No_spending']==1)
    family_spending = (dataset['Family_size']>1) & (dataset['No_spending']==0)
    
    print(f"Numero di persone con Age nullo: {str(dataset['Age'].isna().sum())}, di cui:")
    print(f"Persone sole: {str(dataset['Age'][solo].isna().sum())}")
    print(f"Persone con famiglia che non spendono: {str(dataset['Age'][family_no_spending].isna().sum())}")   
    print(f"Persone con famiglia che spendono: {str(dataset['Age'][family_spending].isna().sum())}")

    dataset['Age'][solo] = dataset[solo]['Age'].transform(lambda x: x.fillna(dataset[solo]['Age'].median()))
    dataset['Age'][family_no_spending] = dataset[family_no_spending]['Age'].transform(lambda x: x.fillna(dataset[family_no_spending]['Age'].median()))
    dataset['Age'][family_spending] = dataset[family_spending]['Age'].transform(lambda x: x.fillna(dataset[family_spending]['Age'].median()))

    print(f"Numero di persone con Age nullo: {str(dataset['Age'].isna().sum())}")   


# Strategia: Surname
# Si osserva che i gruppi sono per gran parte composti da persone con lo stesso cognome, quindi si può assegnare la moda del cognome del gruppo
# alle persone con cognome nullo. Per i gruppi composti da una sola persona si assegna cognome 'unknown'.
def mv_surname(dataset):
    # Raggruppare il dataset per gruppo e contare il numero di cognomi unici
    #TODO: print sbagliata
    group_surname = dataset[(dataset['Solo']==0)].groupby('Group')['Surname'].nunique().value_counts()
    print(f"Numero cognomi diversi per gruppo {group_surname}")

    print(f"Numero di persone con Surname nullo: {str(dataset['Surname'].isna().sum())}")
    dataset['Surname'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('Group')['Surname'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['Surname'][(dataset['Solo']==1)] = dataset[(dataset['Solo']==1)].groupby('Group')['Surname'].transform(lambda x: x.fillna("Unknown"))
    print(f"Numero di persone con Surname nullo: {str(dataset['Surname'].isna().sum())}")

# Strategia: RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
# Si sostituiscono i valori nulli con la media di spesa delle altre categorie non nulle, dopo aver osservato che ogni persona ha almeno un valore
# di spesa non nullo. Ad esempio, se una persona ha spesa nulla per RoomService e ShoppingMall, ma ha spesa per FoodCourt e Spa, si assegna la media
# di FoodCourt e Spa per RoomService e ShoppingMall.
def mv_exp(dataset):
    services = (dataset['RoomService'].isna()) | (dataset['FoodCourt'].isna()) | (dataset['ShoppingMall'].isna()) | (dataset['Spa'].isna()) | (dataset['VRDeck'].isna())
    print(f"Persone con servizi nulli: {str(dataset[services].shape[0])}")
    # Per ogni persona con almeno un servizio nullo, si calcola la media delle spese non nulle
    dataset['RoomService'] = dataset['RoomService'].fillna(dataset[['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].mean(axis=1))
    dataset['FoodCourt'] = dataset['FoodCourt'].fillna(dataset[['RoomService', 'ShoppingMall', 'Spa', 'VRDeck']].mean(axis=1))
    dataset['ShoppingMall'] = dataset['ShoppingMall'].fillna(dataset[['RoomService', 'FoodCourt', 'Spa', 'VRDeck']].mean(axis=1))
    dataset['Spa'] = dataset['Spa'].fillna(dataset[['RoomService', 'FoodCourt', 'ShoppingMall', 'VRDeck']].mean(axis=1))
    dataset['VRDeck'] = dataset['VRDeck'].fillna(dataset[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa']].mean(axis=1))

# Controllare se i gruppi hanno tutti lo stesso valore di cabin_side
def check_cabin_side(dataset):
    # Raggruppa il dataset per 'Group' e conta i valori unici di 'Cabin_side'
    group_cabin_side_counts = dataset.groupby('Group')['Cabin_side'].nunique().value_counts()
    
    # Visualizza il risultato
    print(group_cabin_side_counts)

In [15]:

def preprocess_data(dataset):
    # Viene aggiunta una colonna per contare il numero di persone nel gruppo di appartenenza e se la persona è da sola
    dataset['Group'] = dataset['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)
    dataset['Group_size']=dataset['Group'].map(lambda x: dataset['Group'].value_counts()[x])
    dataset['Solo']=(dataset['Group_size']==1).astype(int)

    # Viene creata una colonna per il cognome e si rimuove la colonna Name
    dataset['Name'].fillna('Unknown Unknown', inplace=True)
    dataset['Surname']=dataset['Name'].str.split().str[-1]
    dataset.loc[dataset['Surname']=='Unknown','Surname']=np.nan
    dataset.drop('Name', axis=1, inplace=True)

    # Vengono riempiti i valori nulli delle colonne RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
    mv_exp(dataset)
    
    # Vengono prese tutte le colonne che rappresentano le spese e si aggiunge una colonna per contare il
    # totale speso dalla persona e una colonna booleana per indicare se la persona non ha speso nulla
    exp_feats=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    dataset['Expenditure']=dataset[exp_feats].sum(axis=1)
    dataset['No_spending']=(dataset['Expenditure']==0).astype(int)

    # Vengono riempiti i valori nulli della colonna HomePlanet, Destination, Surmane
    mv_hp_d(dataset)
    mv_surname(dataset)

    # Viene aggiunta una colonna per contare le persone che hanno lo stesso cognome
    dataset['Family_size']=dataset[(dataset['Surname']!='Unknown')].groupby('Group')['Surname'].transform('count')
    dataset.loc[dataset['Surname']=='Unknown','Family_size']=1
    dataset.loc[dataset['Family_size']>100,'Family_size']=np.nan

    # Vengono riempiti i valori nulli della colonna Age
    mv_age(dataset)


    # Viene aggiunta una colonna che assegna ogni passeggero al gruppo di età di appartenenza
    dataset['Age_group']=0
    dataset.loc[dataset['Age']<=12,'Age_group']='Age_0-12'
    dataset.loc[(dataset['Age']>12) & (dataset['Age']<18),'Age_group']='Age_13-17'
    dataset.loc[(dataset['Age']>=18) & (dataset['Age']<=25),'Age_group']='Age_18-25'
    dataset.loc[(dataset['Age']>25) & (dataset['Age']<=30),'Age_group']='Age_26-30'
    dataset.loc[(dataset['Age']>30) & (dataset['Age']<=50),'Age_group']='Age_31-50'
    dataset.loc[dataset['Age']>50,'Age_group']='Age_51+'

    # Separazione della colonna 'Cabin' nelle colonne deck, number e side
    dataset['Cabin'].fillna('Z/9999/Z', inplace=True)

    dataset['Cabin_deck'] = dataset['Cabin'].apply(lambda x: x.split('/')[0])
    dataset['Cabin_number'] = dataset['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
    dataset['Cabin_side'] = dataset['Cabin'].apply(lambda x: x.split('/')[2])

    dataset.loc[dataset['Cabin_deck']=='Z', 'Cabin_deck']=np.nan
    dataset.loc[dataset['Cabin_number']==9999, 'Cabin_number']=np.nan
    dataset.loc[dataset['Cabin_side']=='Z', 'Cabin_side']=np.nan

    dataset.drop('Cabin', axis=1, inplace=True)
    check_cabin_side(dataset)



preprocess_data(train_df)
#preprocess_data(val_df)

Persone con servizi nulli: 691
Numero di persone con HomePlanet nullo: 168
Numero di gruppi con HomePlanet nullo: 64
Numero di persone da sole con HomePlanet nullo: 104
Tutti gli appartenenti allo stesso gruppo hanno stesso home planet? True
Group          2     3     4     5     6     8     9     10    12    14    \
Destination                                                                 
55 Cancri e       0     0     0     0     0     2     0     0     0     1   
PSO J318.5-22     0     0     0     1     0     0     0     0     0     0   
TRAPPIST-1e       1     2     1     0     2     1     1     1     1     0   

Group          ...  9264  9267  9268  9270  9272  9274  9275  9278  9279  9280  
Destination    ...                                                              
55 Cancri e    ...     1     0     0     1     0     0     0     0     0     0  
PSO J318.5-22  ...     0     0     0     0     0     0     0     1     0     0  
TRAPPIST-1e    ...     0     2     1     0     1

In [16]:
# stampare le colonne con i valori nulli
print(train_df.isnull().sum())

PassengerId       0
HomePlanet        0
CryoSleep       177
Destination       0
Age               0
VIP             162
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Transported       0
Group             0
Group_size        0
Solo              0
Surname           0
Expenditure       0
No_spending       0
Family_size       0
Age_group         0
Cabin_deck      158
Cabin_number    158
Cabin_side      158
dtype: int64
